In [ ]:
################### GPU 이용시 설치 ###################

# 1. 기존 버전 제거 (충돌 방지)
!pip uninstall -y torch torchvision torchaudio torch-geometric torch-scatter torch-sparse

# 2. PyTorch 2.4.1 + CUDA 12.1 설치
# (Colab의 기본 CUDA 버전과 맞추는 것이 좋습니다)
!pip install torch==2.4.1+cu121 torchvision==0.19.1+cu121 torchaudio==2.4.1+cu121 --index-url https://download.pytorch.org/whl/cu121

# 3. PyG 의존성 설치 (버전 매칭 필수: torch-2.4.0+cu121)
# 주의: Torch가 2.4.1이어도 PyG wheel은 보통 2.4.0 경로를 공유합니다.
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

# 4. PyG 메인 설치
!pip install torch-geometric

# [중요] 설치 후 런타임 재시작이 필요할 수 있습니다.
import torch
print(f"Torch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

In [1]:
################### CPU 이용시 설치 ###################
# 1. 기존 제거
!pip uninstall -y torch torchvision torchaudio torch-geometric torch-scatter torch-sparse

# 2. PyTorch 2.4.1 (CPU 전용) 설치
# --index-url을 지정하여 CPU 전용 가벼운 바이너리를 받습니다.
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cpu

# 3. PyG 의존성 설치 (CPU용 Wheel)
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cpu.html

# 4. PyG 메인 설치
!pip install torch-geometric
import torch
print(f"Torch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 

In [2]:
import torch
import torch.nn as nn
from torch_geometric.nn import TransformerConv
from torch_geometric.nn.models.tgn import (
    TGNMemory,
    IdentityMessage,
    LastAggregator
)
import numpy as np

# 장치 설정 (Colab 런타임 유형에 따라 자동 변경)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 하이퍼파라미터 설정
num_nodes = 100
num_events = 1000  # 전체 이벤트 수
dim_msg = 16       # 메시지 벡터 차원
dim_memory = 32    # 메모리 차원
dim_emb = 32       # 임베딩 차원
dim_time = 32      # 시간 인코딩 차원

Using device: cpu


In [84]:
# 가상의 이벤트 데이터 생성 (Source -> Destination, Time, Message)
# 시간(t)은 반드시 오름차순이어야 하므로 정렬합니다.
src = torch.randint(0, num_nodes, (num_events,)).to(device)
dst = torch.randint(0, num_nodes, (num_events,)).to(device)
t = torch.sort((torch.rand(num_events) * 1000).long())[0].to(device) #TGN은 기본적으로 시간을 long으로 정의
msg = torch.randn(num_events, dim_msg).to(device)

# 학습/테스트 분할 (8:2)
split = int(num_events * 0.8)

train_data = {
    'src': src[:split],
    'dst': dst[:split],
    't': t[:split],
    'msg': msg[:split]
}

test_data = {
    'src': src[split:],
    'dst': dst[split:],
    't': t[split:],
    'msg': msg[split:]
}

print(f"Train events: {len(train_data['src'])}, Test events: {len(test_data['src'])}")

Train events: 800, Test events: 200


In [85]:
#https://pytorch-geometric.readthedocs.io/en/2.7.0/_modules/torch_geometric/nn/models/tgn.html
#https://github.com/pyg-team/pytorch_geometric/blob/master/examples/tgn.py
class ToyTGN(nn.Module):
    def __init__(self, num_nodes, raw_msg_dim, memory_dim, time_dim, embedding_dim):
        super().__init__()

        # (A) 메모리 모듈: 노드의 과거 상태 저장
        self.memory = TGNMemory(
            num_nodes=num_nodes,
            raw_msg_dim=raw_msg_dim,
            memory_dim=memory_dim,
            time_dim=time_dim,
            message_module=IdentityMessage(raw_msg_dim, memory_dim, time_dim),
            aggregator_module=LastAggregator()
        )

        # (B) 그래프 임베딩 레이어 (TransformerConv)
        self.gnn = TransformerConv(
            in_channels=memory_dim,
            out_channels=embedding_dim,
            heads=2,
            concat=False
        )

        # (C) 링크 예측기 (Decoder)
        self.link_pred = nn.Sequential(
            nn.Linear(embedding_dim * 2, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, 1)
        )

    def forward(self, src, dst, t, msg):
        # 1. 최신 메모리 상태 조회 (Look-up)
        z_src = self.memory(src)[0]
        z_dst = self.memory(dst)[0]

        # 2. 임베딩 생성 (간소화: Neighbor Sampling 없이 메모리 직접 사용)
        # 빈 edge_index를 넣어 self-loop 개념으로 처리
        empty_edge = torch.zeros((2, 0), dtype=torch.long, device=device)
        emb_src = self.gnn(z_src, edge_index=empty_edge)
        emb_dst = self.gnn(z_dst, edge_index=empty_edge)

        # 3. 링크 예측 (확률값 계산)
        pred = self.link_pred(torch.cat([emb_src, emb_dst], dim=1))

        # 4. 메모리 업데이트 (현재 배치의 정보를 기록)
        print(src.dtype, dst.dtype, t.dtype, msg.dtype)
        self.memory.update_state(src, dst, t, msg)


        return pred

In [86]:
model = ToyTGN(num_nodes, dim_msg, dim_memory, dim_time, dim_emb).to(device)

# TGNMemory의 last_update 버퍼가 float 타입이어야 하는데 long으로 초기화되는 버그 수정
# (torch_geometric 2.7.0 버전에서 발생하는 문제로 추정)
model.memory.last_update = model.memory.last_update.to(torch.float)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

print("Model initialized.")

Model initialized.


In [87]:
print(">>> Start Training")

# 배치 사이즈 설정
batch_size = 200

for epoch in range(10): # 10 Epoch
    total_loss = 0

    # [중요] 에폭 시작 시 메모리를 초기화하거나,
    # BPTT(Backprop Through Time)를 끊어줘야 함 (detach)
    model.memory.reset_state()  # 여기서는 매 에폭 리셋 전략 사용

    # 데이터 로딩
    src_tr, dst_tr = train_data['src'], train_data['dst']
    t_tr, msg_tr = train_data['t'], train_data['msg']

    model.train()

    for i in range(0, len(src_tr), batch_size):
        optimizer.zero_grad()

        # 배치 데이터 슬라이싱
        b_src = src_tr[i:i+batch_size]
        b_dst = dst_tr[i:i+batch_size]
        b_t = t_tr[i:i+batch_size]
        b_msg = msg_tr[i:i+batch_size]

        # 1. Forward (Positive Samples)
        pos_pred = model(b_src, b_dst, b_t, b_msg)

        # 2. Negative Sampling (간단히 랜덤 목적지 생성)
        # 실제로는 TGNMemory 충돌 방지를 위해 더 정교한 처리가 필요하나 Toy에서는 생략
        with torch.no_grad():
            neg_dst = torch.randint(0, num_nodes, (len(b_src),)).to(device)

        # Negative에 대해서도 forward를 태워야 점수가 나옴
        # (주의: update_state가 두 번 호출되면 안 되므로, 실제 구현에선 memory update를 분리해야 함.
        # 여기서는 간단히 pos_pred 계산 시에만 update되었다고 가정)
        neg_pred = model.link_pred(
            torch.cat([model.gnn(model.memory(b_src)[0], torch.zeros((2,0), dtype=torch.long, device=device)),
                       model.gnn(model.memory(neg_dst)[0], torch.zeros((2,0), dtype=torch.long, device=device))], dim=1)
        )

        # 3. Loss Calculation
        loss = criterion(pos_pred, torch.ones_like(pos_pred)) + \
               criterion(neg_pred, torch.zeros_like(neg_pred))

        # 4. Backward
        loss.backward()
        optimizer.step()

        # [중요] 메모리 그래디언트 끊기 (다음 배치를 위해)
        model.memory.detach()

        total_loss += loss.item()

    print(f"Epoch {epoch+1:02d}, Loss: {total_loss:.4f}")

print(">>> Training Finished")

>>> Start Training
torch.int64 torch.int64 torch.int64 torch.float32


RuntimeError: Index put requires the source and destination dtypes match, got Float for the destination and Long for the source.